In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import os
import sys
pd.set_option('display.max_columns',None)

In [ ]:
data = pd.read_csv('Breakfast_diary.csv',engine='python',index_col=0, error_bad_lines=False)
del data['V1']
data.reset_index(drop = True, inplace = True)

In [ ]:
data['season'] = data['NSEASON'].str.split("년 ").str[1].str.split("(").str[0]
data['season'] = data['season'].map(lambda x: x.strip())
data

In [ ]:
data[data['season'].isin(['여름'])]

In [ ]:
#응답자의 demo데이터 
#총 응답자수: 12604명 (남자:6071, 여자:6533 / 20대:2773, 30대: 3149, 4O대: 3469, 50대:3127, 60대:82...)
demo = DataFrame(data, columns=['CUSTNO', 'SEX', 'AGE', 'AGE_5', 'AGE_10', 'AREA', 'NMAR', 'NFAM', 'NCHI', 'NJOB', 'NINC'])
demo = demo.sort_values(by='CUSTNO')
demo = demo.drop_duplicates()
demo.to_csv('demo.csv',index=False,encoding='CP949')

In [ ]:
#취식상황 데이터 
occasion = DataFrame(data, columns=['Q1', 'Q2', 'NWAVE', 'NSEASON', 'NHALF', 'WEEK', 'Q4', 'Q5', 'place', 'season' ])
occasion = data.sort_values(by='ANSWERNO')
occasion = occasion.drop_duplicates()
occasion.to_csv('occasion_new.csv',index=False,encoding='CP949')

In [ ]:
len(demo.loc[demo['SEX'].isin(['남자']), :])

In [ ]:
data['detail'] = data['detail'].map(lambda x: x.strip())
data['middle'] = data['middle'].map(lambda x: x.strip())

In [ ]:
home = data[data['place'].isin(['집'])]
out = data[data['place'].isin(['집밖'])]
dine = data[data['place'].isin(['외식'])]

In [ ]:
home_detail = pd.pivot_table(home,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_detail = home_detail.fillna(0)
home_detail = home_detail.astype(bool)
out_detail = pd.pivot_table(out,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_detail = out_detail.fillna(0)
out_detail = out_detail.astype(bool)
dine_detail = pd.pivot_table(dine,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_detail = dine_detail.fillna(0)
dine_detail = dine_detail.astype(bool)
home_집에서만듦 = home[home['Q36'].isin(['집에서 만듦'])]
home_사먹음 = home[home['Q36'].isin(['마트/온라인 등에서 신선식품/포장식품을 구입하여 그대로 먹음','마트/온라인 등에서 완조리된 음식을 구입하여 먹음','포장간편식을 구입하여 그대로, 혹은 데우거나 간단히 조리하여 먹음'])]
out_집에서만듦 = out[out['Q36'].isin(['집에서 만듦'])]
out_사먹음 = out[out['Q36'].isin(['마트/온라인 등에서 신선식품/포장식품을 구입하여 그대로 먹음','포장간편식을 구입하여 그대로, 혹은 데우거나 간단히 조리하여 먹음', '식당/카페 등에서 테이크아웃'])]
home_집에서만듦_detail = pd.pivot_table(home_집에서만듦,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_집에서만듦_detail = home_집에서만듦_detail.fillna(0)
home_집에서만듦_detail = home_집에서만듦_detail.astype(bool)
home_사먹음_detail = pd.pivot_table(home_사먹음,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_사먹음_detail = home_사먹음_detail.fillna(0)
home_사먹음_detail = home_사먹음_detail.astype(bool)
out_집에서만듦_detail = pd.pivot_table(out_집에서만듦,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_집에서만듦_detail = out_집에서만듦_detail.fillna(0)
out_집에서만듦_detail = out_집에서만듦_detail.astype(bool)
out_사먹음_detail = pd.pivot_table(out_사먹음,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_사먹음_detail = out_사먹음_detail.fillna(0)
out_사먹음_detail = out_사먹음_detail.astype(bool)

In [ ]:
home_ar =(apriori(home_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_ar =(apriori(out_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_ar =(apriori(dine_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)

In [ ]:
home_사먹음_ar =(apriori(home_사먹음_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_사먹음_ar =(apriori(out_사먹음_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_집에서만듦_ar =(apriori(home_집에서만듦_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_집에서만듦_ar =(apriori(out_집에서만듦_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)

In [ ]:
home_ar.to_csv('2.1.1b.csv',index=False,encoding='CP949')
out_ar.to_csv('2.1.2b.csv',index=False,encoding='CP949')
dine_ar.to_csv('2.1.3b.csv',index=False,encoding='CP949')

In [ ]:
home_집에서만듦_ar.to_csv('2.2.1b.csv',index=False,encoding='CP949')
out_집에서만듦_ar.to_csv('2.2.2b.csv',index=False,encoding='CP949')
home_사먹음_ar.to_csv('2.2.3b.csv',index=False,encoding='CP949')
out_사먹음_ar.to_csv('2.2.4b.csv',index=False,encoding='CP949')

In [ ]:
home_weekday = home[home['WEEK'].isin(['월', '화', '수', '목', '금'])]
out_weekday = out[out['WEEK'].isin(['월', '화', '수', '목', '금'])]
dine_weekday = dine[dine['WEEK'].isin(['월', '화', '수', '목', '금'])]
home_weekend = home[home['WEEK'].isin(['토', '일'])]
out_weekend = out[out['WEEK'].isin(['토', '일'])]
dine_weekend = dine[dine['WEEK'].isin(['토', '일'])]
home_weekday_detail = pd.pivot_table(home_weekday,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_weekday_detail = home_weekday_detail.fillna(0)
home_weekday_detail = home_weekday_detail.astype(bool)
out_weekday_detail = pd.pivot_table(out_weekday,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_weekday_detail = out_weekday_detail.fillna(0)
out_weekday_detail = out_weekday_detail.astype(bool)
dine_weekday_detail = pd.pivot_table(dine_weekday,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_weekday_detail = dine_weekday_detail.fillna(0)
dine_weekday_detail = dine_weekday_detail.astype(bool)
#####
home_weekend_detail = pd.pivot_table(home_weekend,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_weekend_detail = home_weekend_detail.fillna(0)
home_weekend_detail = home_weekend_detail.astype(bool)
out_weekend_detail = pd.pivot_table(out_weekend,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_weekend_detail = out_weekend_detail.fillna(0)
out_weekend_detail = out_weekend_detail.astype(bool)
dine_weekend_detail = pd.pivot_table(dine_weekend,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_weekend_detail = dine_weekend_detail.fillna(0)
dine_weekend_detail = dine_weekend_detail.astype(bool)

In [ ]:
home_weekday_ar =(apriori(home_weekday_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_weekday_ar =(apriori(out_weekday_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_weekday_ar =(apriori(dine_weekday_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_weekend_ar =(apriori(home_weekend_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_weekend_ar =(apriori(out_weekend_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_weekend_ar =(apriori(dine_weekend_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)

In [ ]:
home_weekday_ar.to_csv('2.3.1b.csv',index=False,encoding='CP949')
out_weekday_ar.to_csv('2.3.2b.csv',index=False,encoding='CP949')
dine_weekday_ar.to_csv('2.3.3b.csv',index=False,encoding='CP949')
home_weekend_ar.to_csv('2.3.4b.csv',index=False,encoding='CP949')
out_weekend_ar.to_csv('2.3.5b.csv',index=False,encoding='CP949')
dine_weekend_ar.to_csv('2.3.6b.csv',index=False,encoding='CP949')

In [ ]:
home_male = home[home['SEX'].isin(['남자'])]
out_male = out[out['SEX'].isin(['남자'])]
dine_male = dine[dine['SEX'].isin(['남자'])]
home_female = home[home['SEX'].isin(['여자'])]
out_female = out[out['SEX'].isin(['여자'])]
dine_female = dine[dine['SEX'].isin(['여자'])]
home_male_detail = pd.pivot_table(home_male,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_male_detail = home_male_detail.fillna(0)
home_male_detail = home_male_detail.astype(bool)
out_male_detail = pd.pivot_table(out_male,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_male_detail = out_male_detail.fillna(0)
out_male_detail = out_male_detail.astype(bool)
dine_male_detail = pd.pivot_table(dine_male,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_male_detail = dine_male_detail.fillna(0)
dine_male_detail = dine_male_detail.astype(bool)
home_female_detail = pd.pivot_table(home_female,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_female_detail = home_female_detail.fillna(0)
home_female_detail = home_female_detail.astype(bool)
out_female_detail = pd.pivot_table(out_female,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_female_detail = out_female_detail.fillna(0)
out_female_detail = out_female_detail.astype(bool)
dine_female_detail = pd.pivot_table(dine_female,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_female_detail = dine_female_detail.fillna(0)
dine_female_detail = dine_female_detail.astype(bool)

In [ ]:
home_male_ar =(apriori(home_male_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_male_ar =(apriori(out_male_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_male_ar =(apriori(dine_male_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_female_ar =(apriori(home_female_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_female_ar =(apriori(out_female_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_female_ar =(apriori(dine_female_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)

In [ ]:
home_male_ar.to_csv('3.1.1b.csv',index=False,encoding='CP949')
out_male_ar.to_csv('3.1.2b.csv',index=False,encoding='CP949')
dine_male_ar.to_csv('3.1.3b.csv',index=False,encoding='CP949')
home_female_ar.to_csv('3.1.4b.csv',index=False,encoding='CP949')
out_female_ar.to_csv('3.1.5b.csv',index=False,encoding='CP949')
dine_female_ar.to_csv('3.1.6b.csv',index=False,encoding='CP949')

In [ ]:
home_spring = home[home['season'].isin(['봄'])]
out_spring = out[out['season'].isin(['봄'])]
dine_spring = dine[dine['season'].isin(['봄'])]
home_summer = home[home['season'].isin(['여름'])]
out_summer = out[out['season'].isin(['여름'])]
dine_summer = dine[dine['season'].isin(['여름'])]
home_fall = home[home['season'].isin(['가을'])]
out_fall = out[out['season'].isin(['가을'])]
dine_fall = dine[dine['season'].isin(['가을'])]
home_winter = home[home['season'].isin(['겨울'])]
out_winter = out[out['season'].isin(['겨울'])]
dine_winter = dine[dine['season'].isin(['겨울'])]

In [ ]:
out_summer

In [ ]:
home_summer

In [ ]:
dine_summer

In [ ]:
home_spring_detail = pd.pivot_table(home_spring,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_spring_detail = home_spring_detail.fillna(0)
home_spring_detail = home_spring_detail.astype(bool)
out_spring_detail = pd.pivot_table(out_spring,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_spring_detail = out_spring_detail.fillna(0)
out_spring_detail = out_spring_detail.astype(bool)
dine_spring_detail = pd.pivot_table(dine_spring,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_spring_detail = dine_spring_detail.fillna(0)
dine_spring_detail = dine_spring_detail.astype(bool)
home_summer_detail = pd.pivot_table(home_summer,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_summer_detail = home_summer_detail.fillna(0)
home_summer_detail = home_summer_detail.astype(bool)
out_summer_detail = pd.pivot_table(out_summer,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_summer_detail = out_summer_detail.fillna(0)
out_summer_detail = out_summer_detail.astype(bool)
dine_summer_detail = pd.pivot_table(dine_summer,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_summer_detail = dine_summer_detail.fillna(0)
dine_summer_detail = dine_summer_detail.astype(bool)

In [ ]:
home_fall_detail = pd.pivot_table(home_fall,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_fall_detail = home_fall_detail.fillna(0)
home_fall_detail = home_fall_detail.astype(bool)
out_fall_detail = pd.pivot_table(out_fall,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_fall_detail = out_fall_detail.fillna(0)
out_fall_detail = out_fall_detail.astype(bool)
dine_fall_detail = pd.pivot_table(dine_fall,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_fall_detail = dine_fall_detail.fillna(0)
dine_fall_detail = dine_fall_detail.astype(bool)
home_winter_detail = pd.pivot_table(home_winter,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_winter_detail = home_winter_detail.fillna(0)
home_winter_detail = home_winter_detail.astype(bool)
out_winter_detail = pd.pivot_table(out_winter,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_winter_detail = out_winter_detail.fillna(0)
out_winter_detail = out_winter_detail.astype(bool)
dine_winter_detail = pd.pivot_table(dine_winter,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_winter_detail = dine_winter_detail.fillna(0)
dine_winter_detail = dine_winter_detail.astype(bool)

In [ ]:
home_spring_ar =(apriori(home_spring_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_spring_ar =(apriori(out_spring_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_spring_ar =(apriori(dine_spring_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_summer_ar =(apriori(home_summer_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_summer_ar =(apriori(out_summer_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_summer_ar =(apriori(dine_summer_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)

In [ ]:
home_summer_ar

In [ ]:
home_fall_ar =(apriori(home_fall_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_fall_ar =(apriori(out_fall_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_fall_ar =(apriori(dine_fall_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_winter_ar =(apriori(home_winter_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_winter_ar =(apriori(out_winter_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_winter_ar =(apriori(dine_winter_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)

In [ ]:
home_spring_ar.to_csv('2.4.1b.csv',index=False,encoding='CP949')
out_spring_ar.to_csv('2.4.2b.csv',index=False,encoding='CP949')
dine_spring_ar.to_csv('2.4.3b.csv',index=False,encoding='CP949')
home_summer_ar.to_csv('2.4.4b.csv',index=False,encoding='CP949')
out_summer_ar.to_csv('2.4.5b.csv',index=False,encoding='CP949')
dine_summer_ar.to_csv('2.4.6b.csv',index=False,encoding='CP949')
home_fall_ar.to_csv('2.4.7b.csv',index=False,encoding='CP949')
out_fall_ar.to_csv('2.4.8b.csv',index=False,encoding='CP949')
dine_fall_ar.to_csv('2.4.9b.csv',index=False,encoding='CP949')
home_winter_ar.to_csv('2.4.10b.csv',index=False,encoding='CP949')
out_winter_ar.to_csv('2.4.11b.csv',index=False,encoding='CP949')
dine_winter_ar.to_csv('2.4.12b.csv',index=False,encoding='CP949')

In [ ]:
home_1 = home[home['NFAM'].isin(['1명(1인가구)'])]
out_1 = out[out['NFAM'].isin(['1명(1인가구)'])]
dine_1 = dine[dine['NFAM'].isin(['1명(1인가구)'])]
home_2 = home[home['NFAM'].isin(['2명'])]
out_2 = out[out['NFAM'].isin(['2명'])]
dine_2 = dine[dine['NFAM'].isin(['2명'])]
home_3 = home[home['NFAM'].isin(['3명', '4명', '5명 이상'])]
out_3 = out[out['NFAM'].isin(['3명', '4명', '5명 이상'])]
dine_3 = dine[dine['NFAM'].isin(['3명', '4명', '5명 이상'])]

In [ ]:
home_1_detail = pd.pivot_table(home_1,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_1_detail = home_1_detail.fillna(0)
home_1_detail = home_1_detail.astype(bool)
out_1_detail = pd.pivot_table(out_1,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_1_detail = out_1_detail.fillna(0)
out_1_detail = out_1_detail.astype(bool)
dine_1_detail = pd.pivot_table(dine_1,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_1_detail = dine_1_detail.fillna(0)
dine_1_detail = dine_1_detail.astype(bool)
home_2_detail = pd.pivot_table(home_2,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_2_detail = home_2_detail.fillna(0)
home_2_detail = home_2_detail.astype(bool)
out_2_detail = pd.pivot_table(out_2,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_2_detail = out_2_detail.fillna(0)
out_2_detail = out_2_detail.astype(bool)
dine_2_detail = pd.pivot_table(dine_2,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_2_detail = dine_2_detail.fillna(0)
dine_2_detail = dine_2_detail.astype(bool)
home_3_detail = pd.pivot_table(home_3,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_3_detail = home_3_detail.fillna(0)
home_3_detail = home_3_detail.astype(bool)
out_3_detail = pd.pivot_table(out_3,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_3_detail = out_3_detail.fillna(0)
out_3_detail = out_3_detail.astype(bool)
dine_3_detail = pd.pivot_table(dine_3,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_3_detail = dine_3_detail.fillna(0)
dine_3_detail = dine_3_detail.astype(bool)

In [ ]:
home_1_ar =(apriori(home_1_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_1_ar =(apriori(out_1_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_1_ar =(apriori(dine_1_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_2_ar =(apriori(home_2_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_2_ar =(apriori(out_2_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_2_ar =(apriori(dine_2_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_3_ar =(apriori(home_3_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_3_ar =(apriori(out_3_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_3_ar =(apriori(dine_3_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)

In [ ]:
home_1_ar.to_csv('3.2.1b.csv',index=False,encoding='CP949')
out_1_ar.to_csv('3.2.2b.csv',index=False,encoding='CP949')
dine_1_ar.to_csv('3.2.3b.csv',index=False,encoding='CP949')
home_2_ar.to_csv('3.2.4b.csv',index=False,encoding='CP949')
out_2_ar.to_csv('3.2.5b.csv',index=False,encoding='CP949')
dine_2_ar.to_csv('3.2.6b.csv',index=False,encoding='CP949')
home_3_ar.to_csv('3.2.7b.csv',index=False,encoding='CP949')
out_3_ar.to_csv('3.2.8b.csv',index=False,encoding='CP949')
dine_3_ar.to_csv('3.2.9b.csv',index=False,encoding='CP949')

In [ ]:
###기타 4명 제외
home_20 = home[home['AGE_10'].isin(['20대'])]
out_20 = out[out['AGE_10'].isin(['20대'])]
dine_20 = dine[dine['AGE_10'].isin(['20대'])]
home_30 = home[home['AGE_10'].isin(['30대'])]
out_30 = out[out['AGE_10'].isin(['30대'])]
dine_30 = dine[dine['AGE_10'].isin(['30대'])]
home_40 = home[home['AGE_10'].isin(['40대'])]
out_40 = out[out['AGE_10'].isin(['40대'])]
dine_40 = dine[dine['AGE_10'].isin(['40대'])]
home_5060 = home[home['AGE_10'].isin(['50대', '60대'])]
out_5060 = out[out['AGE_10'].isin(['50대', '60대'])]
dine_5060 = dine[dine['AGE_10'].isin(['50대', '60대'])]

In [ ]:
home_20_detail = pd.pivot_table(home_20,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_20_detail = home_20_detail.fillna(0)
home_20_detail = home_20_detail.astype(bool)
out_20_detail = pd.pivot_table(out_20,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_20_detail = out_20_detail.fillna(0)
out_20_detail = out_20_detail.astype(bool)
dine_20_detail = pd.pivot_table(dine_20,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_20_detail = dine_20_detail.fillna(0)
dine_20_detail = dine_20_detail.astype(bool)
home_30_detail = pd.pivot_table(home_30,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_30_detail = home_30_detail.fillna(0)
home_30_detail = home_30_detail.astype(bool)
out_30_detail = pd.pivot_table(out_30,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_30_detail = out_30_detail.fillna(0)
out_30_detail = out_30_detail.astype(bool)
dine_30_detail = pd.pivot_table(dine_30,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_30_detail = dine_30_detail.fillna(0)
dine_30_detail = dine_30_detail.astype(bool)
home_40_detail = pd.pivot_table(home_40,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_40_detail = home_40_detail.fillna(0)
home_40_detail = home_40_detail.astype(bool)
out_40_detail = pd.pivot_table(out_40,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_40_detail = out_40_detail.fillna(0)
out_40_detail = out_40_detail.astype(bool)
dine_40_detail = pd.pivot_table(dine_40,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_40_detail = dine_40_detail.fillna(0)
dine_40_detail = dine_40_detail.astype(bool)
home_5060_detail = pd.pivot_table(home_5060,index=['ANSWERNO'],columns='detail',values='CUSTNO')
home_5060_detail = home_5060_detail.fillna(0)
home_5060_detail = home_5060_detail.astype(bool)
out_5060_detail = pd.pivot_table(out_5060,index=['ANSWERNO'],columns='detail',values='CUSTNO')
out_5060_detail = out_5060_detail.fillna(0)
out_5060_detail = out_5060_detail.astype(bool)
dine_5060_detail = pd.pivot_table(dine_5060,index=['ANSWERNO'],columns='detail',values='CUSTNO')
dine_5060_detail = dine_5060_detail.fillna(0)
dine_5060_detail = dine_5060_detail.astype(bool)

In [ ]:
home_20_ar =(apriori(home_20_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_20_ar =(apriori(out_20_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_20_ar =(apriori(dine_20_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_30_ar =(apriori(home_30_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_30_ar =(apriori(out_30_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_30_ar =(apriori(dine_30_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_40_ar =(apriori(home_40_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_40_ar =(apriori(out_40_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_40_ar =(apriori(dine_40_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
home_5060_ar =(apriori(home_5060_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
out_5060_ar =(apriori(out_5060_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)
dine_5060_ar =(apriori(dine_5060_detail, min_support=0.01, use_colnames=True)).sort_values(by=['support'], axis=0, ascending=False)

In [ ]:
home_20_ar.to_csv('3.3.1b.csv',index=False,encoding='CP949')
out_20_ar.to_csv('3.3.2b.csv',index=False,encoding='CP949')
dine_20_ar.to_csv('3.3.3b.csv',index=False,encoding='CP949')
home_30_ar.to_csv('3.3.4b.csv',index=False,encoding='CP949')
out_30_ar.to_csv('3.3.5b.csv',index=False,encoding='CP949')
dine_30_ar.to_csv('3.3.6b.csv',index=False,encoding='CP949')
home_40_ar.to_csv('3.3.7b.csv',index=False,encoding='CP949')
out_40_ar.to_csv('3.3.8b.csv',index=False,encoding='CP949')
dine_40_ar.to_csv('3.3.9b.csv',index=False,encoding='CP949')
home_5060_ar.to_csv('3.3.10b.csv',index=False,encoding='CP949')
out_5060_ar.to_csv('3.3.11b.csv',index=False,encoding='CP949')
dine_5060_ar.to_csv('3.3.12b.csv',index=False,encoding='CP949')

In [ ]:
data.to_csv('data.csv',index=False,encoding='CP949')